## Import Modules


In [8]:
import arcpy
import os
import sys
arcpy.env.overwriteOutput = True
path = os.getcwd()
print (path)
print (sys.base_exec_prefix)

C:\Users\dwmerrigan\Documents\Github\AKSSF\data_preparation\sensitivity_drivers\geomorphology
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3


## Set working/output directories
1. Add cat_ID_con field to cats_intersect if it doesn't already exist
2. Collect all cats_intersect to merge

In [9]:
# Set data directories and output locations

# Set AKSSF Data directory
data_dir = r"D:\\GIS\\AKSSF"

# Set regional datasets
nhdplus_dat = ['Cook_Inlet','Copper_River']
tauDem_dat = ['Bristol_Bay', 'Kodiak', 'Prince_William_Sound']

# List to store cat_intersect paths
cats = []

# Unique identifier field
unfid = 'cat_ID_con'

# Set workspace to region folder
arcpy.env.workspace = data_dir

# Walk through data dir and collect cat_ints
walk = arcpy.da.Walk(data_dir, datatype = 'FeatureClass')
for dirpath, dirnames, filenames in walk:
    for filename in filenames:
        if filename == 'cats_intersect':
            cat_int = os.path.join(dirpath, filename)
            region = os.path.basename(dirpath)[:-4]
            if region in nhdplus_dat:
                valfield = 'NHDPlusID'
            else:
                valfield = 'catID'
            print(f'Dirpath:{dirpath}\n Filename:{filename}\n  Region:{region}')
            fields = []
            for field in arcpy.ListFields(cat_int):
                fields.append(field.name)
            if 'cat_ID_con' not in fields:
                print(f'unique id field not present adding field {unfid}')
                arcpy.AddField_management(cat_int,unfid,'TEXT')
                # Update 'cat_ID_con' field
                with arcpy.da.UpdateCursor(cat_int, [valfield, unfid]) as cur:
                    for row in cur:
                        strval = str(row[0])
                        row[1] = region + "_" + strval.replace(".0","")
                        # Update
                        cur.updateRow(row)
                    del(row)
                del(cur)
            else:
                print(f'{unfid} field already present')
            cats.append(cat_int)
            print('----------')

Dirpath:D:\GIS\AKSSF\Bristol_Bay\Bristol_Bay.gdb
 Filename:cats_intersect
  Region:Bristol_Bay
cat_ID_con field already present
----------
Dirpath:D:\GIS\AKSSF\Cook_Inlet\Cook_Inlet.gdb
 Filename:cats_intersect
  Region:Cook_Inlet
cat_ID_con field already present
----------
Dirpath:D:\GIS\AKSSF\Copper_River\Copper_River.gdb
 Filename:cats_intersect
  Region:Copper_River
cat_ID_con field already present
----------
Dirpath:D:\GIS\AKSSF\Kodiak\Kodiak.gdb
 Filename:cats_intersect
  Region:Kodiak
cat_ID_con field already present
----------
Dirpath:D:\GIS\AKSSF\Prince_William_Sound\Prince_William_Sound.gdb
 Filename:cats_intersect
  Region:Prince_William_Sound
cat_ID_con field already present
----------


In [14]:
import arcpy,os,time,datetime
# Merge all cats_intersect together
all_cats = arcpy.Merge_management(cats,r'memory\all_cats')

#Set output location for catchment centroid points fc
outgdb = r"D:\\GIS\\AKSSF_land_met\\AKSSF_land_met.gdb"

# Set cat_points output
cat_points = os.path.join(outgdb,'cat_ints_merge_centroid')

# Set coastline fc
coast = r"T:\\Aquatic\\AKSSF\\AKSSF_Hydrography.gdb\\AKSSF_NHD_Coastline"

# Start timing function
start = datetime.datetime.now()
# Convert cats_int to centroid points and generate near table - set INSIDE option
print(f'Converting {cat_int} to centroid point fc {start}...')
cat_points = arcpy.FeatureToPoint_management(all_cats, cat_points, 'INSIDE')
print(f'Getting distance to coast {datetime.datetime.now()}...')
arcpy.analysis.Near(cat_points, coast, None, "NO_LOCATION", "NO_ANGLE", "GEODESIC", "NEAR_DIST NEAR_DIST")
arcpy.AlterField_management(cat_points,'NEAR_DIST','dist_catch_coast_km','dist_catch_coast_km' )

# Convert distance in meters to km
with arcpy.da.UpdateCursor(cat_points,['dist_catch_coast_km']) as cur:
    for row in cur:
        row[0] = row[0] * 0.001
        cur.updateRow(row)
    del(row)
del(cur)

stop = datetime.datetime.now()
elapsed  = stop - start
print (f'Process complete at {datetime.datetime.now()} time elapsed: {elapsed} ')


Converting D:\GIS\AKSSF\Prince_William_Sound\Prince_William_Sound.gdb\cats_intersect to centroid point fc 2022-01-19 10:44:59.816312...
Getting distance to coast 2022-01-19 10:45:01.064321...
Process complete at 2022-01-19 10:45:37.636848 time elapsed: 0:00:37.820536 


### Examine output as pd dataframe

In [15]:
import pandas as pd
import os, arcpy

# Unique identifier field
unfid = 'cat_ID_con'

#Set output location for catchment centroid points fc
outgdb = r"D:\\GIS\\AKSSF_land_met\\AKSSF_land_met.gdb"

# Set cat_points output
cat_points = os.path.join(outgdb,'cat_ints_merge_centroid')

pd.options.display.float_format = '{:.2f}'.format # only display 2 decimal places

# list to store covariate data frames
dfs = []
# Make catchment points df
cat_df = pd.DataFrame()
cat_field_list = []

for field in arcpy.ListFields(cat_points):
    cat_field_list.append(field.name)
cat_arr = arcpy.da.TableToNumPyArray(cat_points, [unfid, 'dist_catch_coast_km'])
cat_df = pd.DataFrame(cat_arr)
cat_df = cat_df.set_index('cat_ID_con')
dfs.append(cat_df)
cat_df

,dist_catch_coast_km
cat_ID_con,
Bristol_Bay_1023044,36.02
Bristol_Bay_2041471,40.57
Bristol_Bay_2065755,23.22
Bristol_Bay_2065914,65.32
Bristol_Bay_2066924,5.09
...,...
Prince_William_Sound_43933,0.33
Prince_William_Sound_43973,0.15
Prince_William_Sound_44553,0.66


In [16]:
import os
#Export CSV to read into R
outdir = r"C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers"
sj_outname = 'AKSSF_dist_catch_coast_km.csv'
arcpy.da.NumPyArrayToTable(cat_arr,os.path.join(outdir,sj_outname))
print('CSV export complete')
print('----------')

CSV export complete
----------
